In [11]:
import requests
from bs4 import BeautifulSoup
import csv
import re

def get_supervisor(work):
    for item in work.find_all('p', class_='vkr-card__item'):
        if "Руководитель:" in item.text:
            supervisor_span = item.find('span', class_='subCell')
            if supervisor_span is not None:
                supervisor_link = supervisor_span.find('a', class_='link')
                if supervisor_link is not None:
                    return supervisor_link.text.strip()
    return 'N/A'

with open('hse_data_2022.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Название ВКР", "Студент", "Полученное образование", "Год", "Оценка", "Руководитель"])

for page in range(1, 118):
    url = f"https://www.hse.ru/edu/vkr/?faculty=122842106&year=2022&page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    works = soup.find_all('li', class_='vkr-card vkr-list__item')

    for work in works:
        title = work.find('a', class_='link').text.strip()
        student = work.find_all('span', class_='vkr-card__value')[1].text.strip()
        program = work.find('span', class_='vkr-card__value').text.strip()
        year = work.find_all('span', class_='vkr-card__value')[-1].text.strip()
        
        grade_tag = work.find('p', class_='vkr-card__item 1')
        grade = grade_tag.find('span', class_='vkr-card__value').text.strip() if grade_tag else 'N/A'
        
        supervisor = get_supervisor(work)

        if grade != 'N/A':
            with open('hse_data_2022.csv', 'a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([title, student, program, year, grade, supervisor])

# Проделываем то же самое для ВКР 2023 года
with open('hse_data_2023.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Название ВКР", "Студент", "Полученное образование", "Год", "Оценка", "Руководитель"])

for page in range(1, 118): 
    url = f"https://www.hse.ru/edu/vkr/?faculty=122842106&year=2023&page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    works = soup.find_all('li', class_='vkr-card vkr-list__item')

    for work in works:
        title = work.find('a', class_='link').text.strip()
        program = work.find('span', class_='vkr-card__value').text.strip()
        student = work.find_all('span', class_='vkr-card__value')[1].text.strip()
        year = work.find_all('span', class_='vkr-card__value')[-1].text.strip()
        
        grade_tag = work.find('p', class_='vkr-card__item 1')
        grade = grade_tag.find('span', class_='vkr-card__value').text.strip() if grade_tag else 'N/A'
        
        supervisor = get_supervisor(work)

        if grade != 'N/A':
            with open('hse_data_2023.csv', 'a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([title, student, program, year, grade, supervisor])


In [14]:
import csv

with open('hse_data_2022.csv', 'r', encoding='utf-8') as file_2022, \
     open('hse_data_2023.csv', 'r', encoding='utf-8') as file_2023:

    data_2022 = csv.reader(file_2022)
    data_2023 = csv.reader(file_2023)

    with open('hse_data_combined.csv', 'w', newline='', encoding='utf-8') as file_combined:
        writer = csv.writer(file_combined)

        headers = next(data_2022)  # Пропускаем заголовок в файле 2022, предполагая, что он такой же, как и в файле 2023
        writer.writerow(headers)

        for row in data_2022:
            writer.writerow(row)

        next(data_2023)

        for row in data_2023:
            writer.writerow(row)

In [19]:
import pandas as pd

data = pd.read_csv("hse_data_combined.csv")
cleaned_data = data.dropna()
cleaned_data.to_csv("hse_vkr_data_2022_2023.csv")
cleaned_data.head(), data.shape[0], cleaned_data.shape[0]

edge_list = cleaned_data[['Студент', 'Руководитель', 'Оценка']]
edge_list.columns = ['source', 'target', 'weight']

edge_list.to_csv('edge_list.csv', index=False)

edge_list.head()

,source,target,weight
0,- Рим Махмуд Лабиб Эльбаати -,Исаев Леонид Маркович,7
1,Айдинович Сергей -,Патоша Ольга Ивановна,8
3,Андреев Сергей Степанович,Зинченко Оксана Олеговна,10
4,Аскар Нана -,Елисеенко Александр Сергеевич,7
5,Аширова Анастасия Леонидовна,Чурилова Елена Владимировна,7
